# Chemical database initialization

## Goal

Bootstrap a chemical database with ~700,000 structures from the US EPA CompTox Dashboard's public dataset.

Set up the database so that it can be used for substructure searching via the [RDKit PostgreSQL database cartridge](http://www.rdkit.org/docs/Cartridge.html).


## Data source

`dsstox_20160701.tsv` - *Mapping file of InChIStrings, InChIKeys and DTXSIDs for the EPA CompTox Dashboard,* [available on Figshare](https://figshare.com/articles/Mapping_file_of_InChIStrings_InChIKeys_and_DTXSIDs_for_the_EPA_CompTox_Dashboard/3578313), published 12.08.2016 by Antony Williams. 
- Date: 2016-07-01
- License: CC0
- Columns: DTXSID (US EPA internal ID), InChI, InChIKey


## Notes on software dependencies

Requires:
- a running instance of PostgreSQL with the RDKit cartridge installed;
- Python packages and dependencies: rdkit, sqlalchemy, psycopg2, pandas.

In [ ]:
import pandas as pd
import csv
from pandas import DataFrame, Series
from rdkit import Chem
from rdkit.Chem import AllChem  #, Draw
# Needs psycopg2 installed
from sqlalchemy import create_engine, types, Table, Column, MetaData
from sqlalchemy.sql import select, text

# Generate database table of structural representations

- Taking the list of EPA InChI(Key)s and DSSTox substance IDs, convert each InChI into a RDKit `Mol` object. Then convert each `Mol` into its binary representation (for reading into PostgreSQL).
- Create a big file that adds a binary molecule-object column to the original EPA dataset. Read that file into PostgreSQL as a table.
    - There is no `mol_from_inchi` method in the PGSQL RDKit extension, otherwise we could do this all in one SQL command.
    - The 720K rows seems to be too much to process in memory all at once, so I am going through the file lazily in chunks.

### Gotchas
- Use all-lowercase column names to avoid SQL mix-ups.
- RDKit will fail to create many of the molecules from InChI because of very specific errors. The number of molecules we have in the end will probably be less than 700K.

In [ ]:
DTX_DATA = '/opt/akokai/data/EPA/dsstox-20160701.tsv'

In [ ]:
!echo "DSSTOX dataset length: $(wc -l /opt/akokai/data/EPA/dsstox-20160701.tsv)"

In [ ]:
# To be able to re-run this code from scratch, first drop the table if it already exists:
!psql chmdata -c 'drop table dtx;'

In [ ]:
conn = create_engine('postgresql://akokai@localhost/chmdata')

In [ ]:
dtypes = {'dtxsid': types.String,
          'inchi': types.String,
          'inchikey': types.String,
          'bin': types.Binary}

dtx = pd.read_table(DTX_DATA, names=['dtxsid', 'inchi', 'inchikey'],
                    chunksize=7200, low_memory=True)

for chunk in dtx:
    chunk['mol'] = chunk.inchi.apply(Chem.MolFromInchi)
    chunk.dropna(inplace=True)
    print(len(chunk), 'molecules created,' 7200 - len(chunk), 'errors')
    chunk['bin'] = chunk.mol.apply(lambda m: m.ToBinary())
    chunk.drop('mol', axis=1, inplace=True)
    chunk.to_sql('dtx', conn, if_exists='append', index=False, chunksize=65536, dtype=dtypes)


In [ ]:

with open(DTX_DATA, 'r', newline='') as epa:
    
    while True:
        lines = [epa.readline() for x in range(7200)]
        if not lines:
            break
        
        reader = csv.reader(lines, delimiter='\t')
        rows = [row for row in reader]
        df = DataFrame(rows, columns=['dtxsid', 'inchi', 'inchikey'])
        df['mols'] = dtx.inchi.apply(Chem.MolFromInchi)
        df.dropna(inplace=True)
        print(len(df), 'molecules created')
        df['bin'] = df.mols.apply(lambda m: m.ToBinary())
        df.drop('mols', axis=1, inplace=True)
        # dtx.to_sql('epa', conn, if_exists='append', index=False, chunksize=65536, dtype=dtypes)

        # Because testing:
        break

df.head()

### Just make sure it's there...
Can delete these cells after table creation & reflection are known to work.

In [ ]:
# Reflect this table in an SQLAlchemy object.
meta_dtx = MetaData()
dtx = Table('dtx', meta_epa, autoload=True, autoload_with=conn)

In [ ]:
[c.name for c in epa.columns]

# Generate `mol`-type column and index structures

Create a new table with `(dtxsid, inchi, ..., molecule)` and index the molecules using the GiST-powered RDKit extension. This is what enables substructure searching in SQL.

In [ ]:
# To be able to re-run the code below, first drop the table:
!psql chmdata -c 'drop table chem;'

In [ ]:
cmd = text(
    '''create table chem
       as select dtxsid, inchi, inchikey, mol_from_pkl(bin) molecule from epa;''')
res = conn.execute(cmd)

In [ ]:
# Test to see if that worked... YES!
cmd = text('select * from chem limit 5;')
conn.execute(cmd).fetchall()

In [ ]:
# Check to see column names.
meta_chem = MetaData()
chem = Table('chem', meta_chem, autoload=True, autoload_with=conn)
[c.name for c in chem.columns]

In [ ]:
# Hopefully this works if it produces no errors...
cmd = text('create index molidx on chem using gist(molecule);')
res = conn.execute(cmd)

# Generate PubChem CIDs for the set of compounds

It will help to have some 'known' ID to integrate results into the CML. This dataset has no CASRN column. Let's try to find CIDs for each InChIKey using the [PubChem ID exchange](https://pubchem.ncbi.nlm.nih.gov/idexchange/idexchange.cgi).
- Note: Could try matching by full InChI string, but it is way slower.

In [ ]:
!echo "EPA dataset length: $(wc -l /opt/akokai/data/EPA/dsstox-20160701.tsv)"

Extract the **InChIKeys**, and split the resulting file into two files with less than 500K lines each (required for ID exchange service).

In [ ]:
!cat /opt/akokai/data/EPA/dsstox-20160701.tsv | awk -F '\t' '{print $3}' > /opt/akokai/data/EPA/dsstox-inchikey.txt
!split -l 360000 /opt/akokai/data/EPA/dsstox-inchikey.txt /opt/akokai/data/EPA/dsstox-inchikey- --additional-suffix .txt

In [ ]:
CIDS_FILES = ['/opt/akokai/data/EPA/dsstox-cid-inchikey-1.txt',
              '/opt/akokai/data/EPA/dsstox-cid-inchikey-2.txt']

cids = pd.concat([pd.read_table(f, names=['inchikey', 'cid'], dtype=str)
                  for f in CIDS_FILES])
cids.dropna(inplace=True)
print('InChIKey-CID mappings:', len(cids))
cids.head()

In [ ]:
import matplotlib
%matplotlib inline
multi = cids.groupby('inchikey')['cid'].count()
multi.hist()

In [ ]:
multi.head()

In [ ]:
cids['multi'] = cids['inchikey'].apply(lambda i: multi[i])

In [ ]:
len(cids[cids.multi == 1])